# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [159]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [166]:
def date_sorter():
    # Your code here
    order = None
    df_ = df.copy()
    # Create a dictionary to map abbreviated and misspelled month names to month numbers
    month_dict = {
        'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
        'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12',
        'January': '1', 'February': '2', 'March': '3', 'April': '4', 'June': '6', 'July': '7', 'August': '8',
        'September': '9', 'October': '10', 'November': '11', 'December': '12', 'Decemeber': '12',
        'Janaury': '1'  # Misspelling corrected
    }

    patterns = [
        # Match dates in the format "MM/DD/YYYY" or "MM-DD-YY"
        r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>(?:\d{4}|\d{2}))\b',

        # Match full month names followed by day and four-digit year, with optional punctuation or spaces in between
        r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)(?:[\-.,]? )(?P<day>\d{2}[a-z]{0,2}),? (?P<year>\d{4})',

        # Match dates where the day precedes the month name and a four-digit year, separated by space and optional punctuation
        r'(?P<day>\d{2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[(?:. )(?:, )](?P<year>\d{4})',

        # Match a single character (alpha-numeric) followed by a month name and optional comma, followed by a year
        r'[A-Za-z0-9]{1}(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),? (?P<year>\d{4})',

        # Match any character except a digit, followed by month name, optional punctuation, and a year
        r'[^0-9],? (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),? (?P<year>\d{4})',

        # Match a digit for the month followed by a slash or non-digit, then a four-digit year
        r'[^/0-9](?P<month>\d{1,2})/(?P<year>\d{4})',

        # Match month names at the start of the string followed by an optional comma and a year
        r'^(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),? (?P<year>\d{4})',

        # Match parentheses, period, or quotation followed by a month name, optional comma, and a year
        r'[\(\."](?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),? (?P<year>\d{4})',

        # Match dates starting with month/day followed by a dash/slash and a four-digit year
        r'^(?P<month>\d{1,2})[/-](?P<year>\d{4})',

        # Match a non-alphanumeric character, a space, and a four-digit year, ensuring it's not preceded or followed by a digit
        r'[^0-9a-z], (?P<year>\d{4})[^0-9]', 

        # Match a four-digit year at the start of the string
        r'^(?P<year>\d{4})',

        # Match a single character (alpha or punctuation), followed by a four-digit year
        r'[A-Za-z\.(~]{1}(?P<year>\d{4})',

        # Match the word 'Age', followed by an optional comma, a 1 or 2-digit number, a comma, and a four-digit year
        r'Age,? \d{1,2}, (?P<year>\d{4})',

        # Match a three-character string that does not start with a month name or certain other strings, followed by a year
        r'(?!Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|ury|ary|rch|ril|une|uly|ust|ber|\d{3}$)[a-zA-Z:;]{3} (?P<year>\d{4})',

        # Match the word 'in' followed by a space and a four-digit year
        r'[Ii]n (?P<year>\d{4})',

        # Match a space, a hyphen, a space, and a four-digit year
        r' - (?P<year>\d{4})',

        # Match any three digits followed by a space and a four-digit year
        r'\d{3} (?P<year>\d{4})'
    ]


    # Extract dates from each text using the regex patterns
    dates = []
    for idx, pattern in enumerate(patterns):
        date = df_.str.extractall(pattern)
        dates.append(date)

    # Combine all the extracted dates into a single dataframe
    dates_df = pd.concat(dates).sort_index()

    # Fill missing day values with '1' and convert to string
    dates_df['day'] = dates_df['day'].fillna(1)
    dates_df['day'] = dates_df['day'].astype('int').astype('str')

    # Fill missing month values with 'January' and replace with month numbers, convert to string
    dates_df['month'] = dates_df['month'].fillna('January')
    dates_df['month'].replace(month_dict, inplace=True)
    dates_df['month'] = dates_df['month'].astype('int').astype('str')

    # Adjust year values (prepend '19' for two-digit years), convert to int, filter out future years, convert back to string
    dates_df['year'] = dates_df['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    dates_df['year'] = dates_df['year'].astype('int')
    dates_df = dates_df[dates_df['year'] <= 2023]
    dates_df['year'] = dates_df['year'].astype('str')

    # Remove the multi-index and assemble the full date string
    extracted_df = dates_df.droplevel(level='match')
    extracted_df['date'] = extracted_df['month'] + '/' + extracted_df['day'] + '/' + extracted_df['year']

    # Convert the date string to datetime object and sort
    times_df = pd.to_datetime(extracted_df['date'])
    order = pd.Series(times_df.sort_values(kind='stable').index)
    
 # Return the sorted series of indices
    return order  # Your answer here
    raise NotImplementedError()


In [167]:
date_sorter()



0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64